# FINAL CAPSTONE ANALYSIS
## Search out Hospital and pharmacy Neighborhoods in Toronto

In [ ]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [5]:
CLIENT_ID = '2VJSZMEZBVCII3HYWBCNGEDFTEI3JAVBB3BU0X1XRGN4LGM1' 
CLIENT_SECRET = '0JYZ0FESUQ02YKG3WUKAAOIHTIMVKHM1TYXYBW1ML3PF4QGB'
VERSION = '20200417'
LIMIT= 50

In [6]:
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [32]:
search_query = 'Hospital'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

result_Hospital = requests.get(url).json()

result_Hospital

venue_Hospital = result_Hospital['response']['venues']

df = pd.json_normalize(venue_Hospital)
df.shape



(27, 18)

In [33]:
new_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')]+ ['id']
new_df = df.loc[:,new_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

new_df['categories'] = new_df.apply(get_category_type, axis=1)

new_df.columns = [column.split('.')[-1] for column in new_df.columns]

new_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,The Hospital for Sick Children (SickKids),Hospital,555 University Ave.,at Gerrard St.,43.657499,-79.386512,"[{'label': 'display', 'lat': 43.65749866896264...",492,M5G 1X8,CA,Toronto,ON,Canada,"[555 University Ave. (at Gerrard St.), Toronto...",NaN,4ad4c064f964a5206ef820e3
1,Toronto General Hospital,Hospital,190 Elizabeth St,at Gerrard St W,43.658762,-79.388292,"[{'label': 'display', 'lat': 43.65876218215914...",684,M5G 2C4,CA,Toronto,ON,Canada,"[190 Elizabeth St (at Gerrard St W), Toronto O...",NaN,4ad4c064f964a52070f820e3
2,St. Michael's Hospital,Hospital,30 Bond St,at Queen St E,43.653784,-79.377809,"[{'label': 'display', 'lat': 43.65378419391942...",494,M5B 1W8,CA,Toronto,ON,Canada,"[30 Bond St (at Queen St E), Toronto ON M5B 1W...",NaN,4ad4c064f964a5206ff820e3
3,Toronto General Hospital Emergency Room,Emergency Room,190 Elizabeth St,at Gerrard St W,43.658426,-79.386462,"[{'label': 'display', 'lat': 43.65842614181627...",586,NaN,CA,Toronto,ON,Canada,"[190 Elizabeth St (at Gerrard St W), Toronto O...",NaN,4e3dd18362e19d6109746a06
4,Hospital Row,None,NaN,NaN,43.653939,-79.380815,"[{'label': 'display', 'lat': 43.65393873543311...",256,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4fd8e916e4b0a8dbd1d82589


In [34]:
new_df= new_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
new_df


,name,categories,address,lat,lng,postalCode,state,neighborhood
0,The Hospital for Sick Children (SickKids),Hospital,555 University Ave.,43.657499,-79.386512,M5G 1X8,ON,NaN
1,Toronto General Hospital,Hospital,190 Elizabeth St,43.658762,-79.388292,M5G 2C4,ON,NaN
2,St. Michael's Hospital,Hospital,30 Bond St,43.653784,-79.377809,M5B 1W8,ON,NaN
3,Toronto General Hospital Emergency Room,Emergency Room,190 Elizabeth St,43.658426,-79.386462,NaN,ON,NaN
4,Hospital Row,None,NaN,43.653939,-79.380815,NaN,NaN,NaN
5,St. Michael's Hospital - Glaucoma & Nerve Prot...,Eye Doctor,NaN,43.653703,-79.379490,NaN,NaN,NaN
6,Inpatient Lounge - St. Michael's Hospital,Hospital,NaN,43.653428,-79.379383,NaN,NaN,NaN
7,MRI Department Sick Kids Hospital,Hospital,555 University Ave.,43.657149,-79.384323,NaN,ON,NaN
8,St Michael Hospital,Hospital,50 Bond street,43.654024,-79.378736,NaN,ON,NaN
9,Rouge Valley Hospital,Hospital,580 Harwood Avenue Soith,43.654843,-79.379067,NaN,ON,NaN


In [57]:
array= ['Hospital']
df_Hospital= new_df.loc[new_df['categories'].isin(array)]
df_Hospital

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,The Hospital for Sick Children (SickKids),Hospital,555 University Ave.,43.657499,-79.386512,M5G 1X8,ON,NaN
1,Toronto General Hospital,Hospital,190 Elizabeth St,43.658762,-79.388292,M5G 2C4,ON,NaN
2,St. Michael's Hospital,Hospital,30 Bond St,43.653784,-79.377809,M5B 1W8,ON,NaN
6,Inpatient Lounge - St. Michael's Hospital,Hospital,NaN,43.653428,-79.379383,NaN,NaN,NaN
7,MRI Department Sick Kids Hospital,Hospital,555 University Ave.,43.657149,-79.384323,NaN,ON,NaN
8,St Michael Hospital,Hospital,50 Bond street,43.654024,-79.378736,NaN,ON,NaN
9,Rouge Valley Hospital,Hospital,580 Harwood Avenue Soith,43.654843,-79.379067,NaN,ON,NaN
10,Endoscopy Unit St Michaels Hospital,Hospital,NaN,43.652371,-79.378923,NaN,NaN,NaN
13,Voice Clinic - St Mike's Hospital,Hospital,Queen & Victoria,43.653706,-79.377975,NaN,ON,NaN
14,St Michael's Hospital - Gastroenterology,Hospital,NaN,43.653653,-79.378081,M5B 1W8,ON,NaN


In [46]:
search_query = '{pharmacy}'
radius = 10000

url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=2VJSZMEZBVCII3HYWBCNGEDFTEI3JAVBB3BU0X1XRGN4LGM1&client_secret=0JYZ0FESUQ02YKG3WUKAAOIHTIMVKHM1TYXYBW1ML3PF4QGB&ll=43.6534817,-79.3839347&v=20200417&query={pharmacy}&radius=10000&limit=50'

In [51]:
result_Pharmacy = requests.get(url2).json()

venue_Pharmacy = result_Pharmacy['response']['venues']

Pharmacy_df = pd.json_normalize(venue_Pharmacy)
Pharmacy_df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,50e60a65e4b002e7385dd61d,I.D.A. - Pharmacy By The Grange,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1587182402,False,275 Dundas St W,43.654158,-79.390490,"[{'label': 'display', 'lat': 43.654158, 'lng':...",533,M5T 1G1,CA,Toronto,ON,Canada,"[275 Dundas St W, Toronto ON M5T 1G1, Canada]",NaN,NaN
1,51bf41d4498ed2fdf40979d5,On Care Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1587182402,False,481 Dundas West,43.653188,-79.397056,"[{'label': 'display', 'lat': 43.6531877185835,...",1057,M5T 1H1,CA,Toronto,ON,Canada,"[481 Dundas West (Spadina), Toronto ON M5T 1H1...",Spadina,NaN
2,5bd0777deaf49a002c0eed78,U & C Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1587182402,False,"700 University Ave,112",43.658808,-79.390989,"[{'label': 'display', 'lat': 43.658808, 'lng':...",821,M5G 1Z5,CA,Toronto,ON,Canada,"[700 University Ave,112, Toronto ON M5G 1Z5, C...",NaN,NaN
3,5440288c498e60c2ff8638da,RK Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1587182402,False,302 Spadina Avenue,43.653262,-79.398402,"[{'label': 'display', 'lat': 43.653262, 'lng':...",1165,NaN,CA,NaN,NaN,Canada,"[302 Spadina Avenue (Dundas Street), Canada]",Dundas Street,Kensington Market
4,4b717e5ef964a52083492de3,Leslie L. Dan Pharmacy Building,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",v-1587182402,False,144 College St.,43.659963,-79.391329,"[{'label': 'display', 'lat': 43.65996261222835...",935,M5S,CA,Toronto,ON,Canada,"[144 College St. (University of Toronto), Toro...",University of Toronto,NaN


In [61]:
Pharmacy_new_columns = ['name', 'categories'] + [col for col in Pharmacy_df.columns if col.startswith('location.')]+ ['id']
new_df_Pharmacy = Pharmacy_df.loc[:,Pharmacy_new_columns]

def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

new_df_Pharmacy['categories'] = new_df_Pharmacy.apply(get_category_type, axis=1)

new_df_Pharmacy.columns = [column.split('.')[-1] for column in new_df_Pharmacy.columns]

new_df_Pharmacy.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,I.D.A. - Pharmacy By The Grange,Pharmacy,275 Dundas St W,43.654158,-79.390490,"[{'label': 'display', 'lat': 43.654158, 'lng':...",533,M5T 1G1,CA,Toronto,ON,Canada,"[275 Dundas St W, Toronto ON M5T 1G1, Canada]",NaN,NaN,50e60a65e4b002e7385dd61d
1,On Care Pharmacy,Pharmacy,481 Dundas West,43.653188,-79.397056,"[{'label': 'display', 'lat': 43.6531877185835,...",1057,M5T 1H1,CA,Toronto,ON,Canada,"[481 Dundas West (Spadina), Toronto ON M5T 1H1...",Spadina,NaN,51bf41d4498ed2fdf40979d5
2,U & C Pharmacy,Pharmacy,"700 University Ave,112",43.658808,-79.390989,"[{'label': 'display', 'lat': 43.658808, 'lng':...",821,M5G 1Z5,CA,Toronto,ON,Canada,"[700 University Ave,112, Toronto ON M5G 1Z5, C...",NaN,NaN,5bd0777deaf49a002c0eed78
3,RK Pharmacy,Pharmacy,302 Spadina Avenue,43.653262,-79.398402,"[{'label': 'display', 'lat': 43.653262, 'lng':...",1165,NaN,CA,NaN,NaN,Canada,"[302 Spadina Avenue (Dundas Street), Canada]",Dundas Street,Kensington Market,5440288c498e60c2ff8638da
4,Leslie L. Dan Pharmacy Building,College Academic Building,144 College St.,43.659963,-79.391329,"[{'label': 'display', 'lat': 43.65996261222835...",935,M5S,CA,Toronto,ON,Canada,"[144 College St. (University of Toronto), Toro...",University of Toronto,NaN,4b717e5ef964a52083492de3


In [62]:
new_df_Pharmacy= new_df_Pharmacy.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
new_df_Pharmacy

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,I.D.A. - Pharmacy By The Grange,Pharmacy,275 Dundas St W,43.654158,-79.390490,M5T 1G1,ON,NaN
1,On Care Pharmacy,Pharmacy,481 Dundas West,43.653188,-79.397056,M5T 1H1,ON,NaN
2,U & C Pharmacy,Pharmacy,"700 University Ave,112",43.658808,-79.390989,M5G 1Z5,ON,NaN
3,RK Pharmacy,Pharmacy,302 Spadina Avenue,43.653262,-79.398402,NaN,NaN,Kensington Market
4,Leslie L. Dan Pharmacy Building,College Academic Building,144 College St.,43.659963,-79.391329,M5S,ON,NaN
5,Guardian - Morelli's Pharmacy,Pharmacy,15 York St,43.642418,-79.380937,M5J 0A3,ON,NaN
6,I.D.A. - The Palmerston Pharmacy & HomeCare,Pharmacy,499 College St,43.655889,-79.409718,M6G 1A5,ON,NaN
7,Pharmacy PMH,Pharmacy,NaN,43.656351,-79.391775,NaN,ON,NaN
8,Guardian Pharmacy,Pharmacy,755 Dundas St. W,43.655781,-79.383859,NaN,ON,NaN
9,Victor Pharmacy,Pharmacy,123 Edward St.,43.655445,-79.386335,NaN,ON,NaN


In [63]:
df_Pharmacy = new_df_Pharmacy.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Pharmacy

,name,categories,address,lat,lng,postalCode,state,neighborhood


In [64]:
neighbourhood_df = pd.concat([df_Hospital, df_Pharmacy], ignore_index=True)
neighbourhood_df

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,The Hospital for Sick Children (SickKids),Hospital,555 University Ave.,43.657499,-79.386512,M5G 1X8,ON,NaN
1,Toronto General Hospital,Hospital,190 Elizabeth St,43.658762,-79.388292,M5G 2C4,ON,NaN
2,St. Michael's Hospital,Hospital,30 Bond St,43.653784,-79.377809,M5B 1W8,ON,NaN
3,Inpatient Lounge - St. Michael's Hospital,Hospital,NaN,43.653428,-79.379383,NaN,NaN,NaN
4,MRI Department Sick Kids Hospital,Hospital,555 University Ave.,43.657149,-79.384323,NaN,ON,NaN
5,St Michael Hospital,Hospital,50 Bond street,43.654024,-79.378736,NaN,ON,NaN
6,Rouge Valley Hospital,Hospital,580 Harwood Avenue Soith,43.654843,-79.379067,NaN,ON,NaN
7,Endoscopy Unit St Michaels Hospital,Hospital,NaN,43.652371,-79.378923,NaN,NaN,NaN
8,Voice Clinic - St Mike's Hospital,Hospital,Queen & Victoria,43.653706,-79.377975,NaN,ON,NaN
9,St Michael's Hospital - Gastroenterology,Hospital,NaN,43.653653,-79.378081,M5B 1W8,ON,NaN


In [68]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, name, categories, address in zip(neighbourhood_df['lat'], neighbourhood_df['lng'], 
                                           neighbourhood_df['name'], neighbourhood_df['categories'],\
                                               neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto